In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("Hello Spark") \
        .master("local[3]") \
        .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/12/06 23:16:03 WARN Utils: Your hostname, otaviordgs resolves to a loopback address: 127.0.1.1; using 172.24.57.217 instead (on interface eth0)
24/12/06 23:16:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/06 23:16:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.types import _parse_datatype_string

schema = """
    Poster_Link STRING,
    Series_Title STRING,
    Released_Year INT,
    Certificate STRING,
    Runtime STRING,
    Genre STRING,
    IMDB_Rating FLOAT,
    Overview STRING,
    Meta_score INT,
    Director STRING,
    Star1 STRING,
    Star2 STRING,
    Star3 STRING,
    Star4 STRING,
    No_of_Votes INT,
    Gross STRING
"""
schema = _parse_datatype_string(schema)
schema

StructType([StructField('Poster_Link', StringType(), True), StructField('Series_Title', StringType(), True), StructField('Released_Year', IntegerType(), True), StructField('Certificate', StringType(), True), StructField('Runtime', StringType(), True), StructField('Genre', StringType(), True), StructField('IMDB_Rating', FloatType(), True), StructField('Overview', StringType(), True), StructField('Meta_score', IntegerType(), True), StructField('Director', StringType(), True), StructField('Star1', StringType(), True), StructField('Star2', StringType(), True), StructField('Star3', StringType(), True), StructField('Star4', StringType(), True), StructField('No_of_Votes', IntegerType(), True), StructField('Gross', StringType(), True)])

In [3]:
df = spark.read \
    .format("cvs") \
    .option('header', 'true') \
    .schema(schema) \
    .csv('data/imdb_top_1000.csv')

df.show()

+--------------------+--------------------+-------------+-----------+-------+--------------------+-----------+--------------------+----------+--------------------+------------------+--------------------+------------------+--------------------+-----------+-----------+
|         Poster_Link|        Series_Title|Released_Year|Certificate|Runtime|               Genre|IMDB_Rating|            Overview|Meta_score|            Director|             Star1|               Star2|             Star3|               Star4|No_of_Votes|      Gross|
+--------------------+--------------------+-------------+-----------+-------+--------------------+-----------+--------------------+----------+--------------------+------------------+--------------------+------------------+--------------------+-----------+-----------+
|https://m.media-a...|The Shawshank Red...|         1994|          A|142 min|               Drama|        9.3|Two imprisoned me...|        80|      Frank Darabont|       Tim Robbins|      Morgan F

In [4]:
df.printSchema()

root
 |-- Poster_Link: string (nullable = true)
 |-- Series_Title: string (nullable = true)
 |-- Released_Year: integer (nullable = true)
 |-- Certificate: string (nullable = true)
 |-- Runtime: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- IMDB_Rating: float (nullable = true)
 |-- Overview: string (nullable = true)
 |-- Meta_score: integer (nullable = true)
 |-- Director: string (nullable = true)
 |-- Star1: string (nullable = true)
 |-- Star2: string (nullable = true)
 |-- Star3: string (nullable = true)
 |-- Star4: string (nullable = true)
 |-- No_of_Votes: integer (nullable = true)
 |-- Gross: string (nullable = true)



In [5]:
from pyspark.sql.functions import pandas_udf, udf

@udf
def get_runtime_minutes(runtime):
    return runtime.split(" ")[0]

In [6]:
# pegar o ano em que os filmes foram lançados
from pyspark.sql.functions import expr

df1 = df.withColumn("Runtime_Minutes", get_runtime_minutes("Runtime"))
df1.show()

+--------------------+--------------------+-------------+-----------+-------+--------------------+-----------+--------------------+----------+--------------------+------------------+--------------------+------------------+--------------------+-----------+-----------+---------------+
|         Poster_Link|        Series_Title|Released_Year|Certificate|Runtime|               Genre|IMDB_Rating|            Overview|Meta_score|            Director|             Star1|               Star2|             Star3|               Star4|No_of_Votes|      Gross|Runtime_Minutes|
+--------------------+--------------------+-------------+-----------+-------+--------------------+-----------+--------------------+----------+--------------------+------------------+--------------------+------------------+--------------------+-----------+-----------+---------------+
|https://m.media-a...|The Shawshank Red...|         1994|          A|142 min|               Drama|        9.3|Two imprisoned me...|        80|      

In [7]:
df1 = df1.drop("Runtime")
df1.show()

+--------------------+--------------------+-------------+-----------+--------------------+-----------+--------------------+----------+--------------------+------------------+--------------------+------------------+--------------------+-----------+-----------+---------------+
|         Poster_Link|        Series_Title|Released_Year|Certificate|               Genre|IMDB_Rating|            Overview|Meta_score|            Director|             Star1|               Star2|             Star3|               Star4|No_of_Votes|      Gross|Runtime_Minutes|
+--------------------+--------------------+-------------+-----------+--------------------+-----------+--------------------+----------+--------------------+------------------+--------------------+------------------+--------------------+-----------+-----------+---------------+
|https://m.media-a...|The Shawshank Red...|         1994|          A|               Drama|        9.3|Two imprisoned me...|        80|      Frank Darabont|       Tim Robbin

In [8]:
df1 = df1.withColumn("Runtime_Minutes", expr("Cast(Runtime_Minutes as INT)"))
df1.printSchema()

root
 |-- Poster_Link: string (nullable = true)
 |-- Series_Title: string (nullable = true)
 |-- Released_Year: integer (nullable = true)
 |-- Certificate: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- IMDB_Rating: float (nullable = true)
 |-- Overview: string (nullable = true)
 |-- Meta_score: integer (nullable = true)
 |-- Director: string (nullable = true)
 |-- Star1: string (nullable = true)
 |-- Star2: string (nullable = true)
 |-- Star3: string (nullable = true)
 |-- Star4: string (nullable = true)
 |-- No_of_Votes: integer (nullable = true)
 |-- Gross: string (nullable = true)
 |-- Runtime_Minutes: integer (nullable = true)



In [9]:
df2 = df1.select("Series_Title", "Runtime_Minutes", expr("""
    CASE
        WHEN Runtime_Minutes < 60 THEN '< 1'
        WHEN Runtime_Minutes >= 60 AND Runtime_Minutes < 90 THEN '>= 1 && < 1,5'
        WHEN Runtime_Minutes >= 90 AND Runtime_Minutes < 120 THEN '>= 1,5 && < 2'
        ELSE '>= 2'
    END AS Runtime_hours
"""))
df2.show()

+--------------------+---------------+-------------+
|        Series_Title|Runtime_Minutes|Runtime_hours|
+--------------------+---------------+-------------+
|The Shawshank Red...|            142|         >= 2|
|       The Godfather|            175|         >= 2|
|     The Dark Knight|            152|         >= 2|
|The Godfather: Pa...|            202|         >= 2|
|        12 Angry Men|             96|>= 1,5 && < 2|
|The Lord of the R...|            201|         >= 2|
|        Pulp Fiction|            154|         >= 2|
|    Schindler's List|            195|         >= 2|
|           Inception|            148|         >= 2|
|          Fight Club|            139|         >= 2|
|The Lord of the R...|            178|         >= 2|
|        Forrest Gump|            142|         >= 2|
|Il buono, il brut...|            161|         >= 2|
|The Lord of the R...|            179|         >= 2|
|          The Matrix|            136|         >= 2|
|          Goodfellas|            146|        

In [10]:
import pyspark.sql.functions as f

df3 = df1.select(
    f.max("Released_Year").alias("most_recent_movie"),
    f.min("Released_Year").alias("less_recent_movie"),
    f.max("IMDB_Rating").alias("highest_note"),
    f.min("IMDB_Rating").alias("lowest_note")
).show()

+-----------------+-----------------+------------+-----------+
|most_recent_movie|less_recent_movie|highest_note|lowest_note|
+-----------------+-----------------+------------+-----------+
|             2020|             1920|         9.3|        7.6|
+-----------------+-----------------+------------+-----------+



In [24]:
df4 = df1.groupBy("Director") \
    .agg(f.max("IMDB_Rating").alias("best"),f.min("IMDB_Rating").alias("worst")) \
    .orderBy(f.desc("best"), f.asc("Director")) \
    .show()

+--------------------+----+-----+
|            Director|best|worst|
+--------------------+----+-----+
|      Frank Darabont| 9.3|  8.6|
|Francis Ford Coppola| 9.2|  7.6|
|   Christopher Nolan| 9.0|  7.8|
|        Sidney Lumet| 9.0|  7.7|
|       Peter Jackson| 8.9|  7.8|
|   Quentin Tarantino| 8.9|  7.6|
|    Steven Spielberg| 8.9|  7.6|
|       David Fincher| 8.8|  7.7|
|     Robert Zemeckis| 8.8|  7.7|
|        Sergio Leone| 8.8|  7.6|
|      Irvin Kershner| 8.7|  8.7|
|      Lana Wachowski| 8.7|  8.7|
|     Martin Scorsese| 8.7|  7.7|
|        Milos Forman| 8.7|  8.3|
|      Akira Kurosawa| 8.6|  8.0|
|        Bong Joon Ho| 8.6|  7.8|
|  Fernando Meirelles| 8.6|  8.6|
|         Frank Capra| 8.6|  8.0|
|        George Lucas| 8.6|  8.6|
|      Hayao Miyazaki| 8.6|  7.7|
+--------------------+----+-----+
only showing top 20 rows



In [30]:
df5 = df1.select("Series_Title") \
    .where(df1.IMDB_Rating >= 8.9) \
    .show()

+--------------------+
|        Series_Title|
+--------------------+
|The Shawshank Red...|
|       The Godfather|
|     The Dark Knight|
|The Godfather: Pa...|
|        12 Angry Men|
+--------------------+

